# Examining The Influence Of Square Footage On The Selling Price Of Houses In Vancouver #

## Introduction: ##

The Vancouver housing market is notorious for its skyrocketing prices-- making it the "second-most unaffordable [housing] market" after Hong Kong (Bloomberg, 2019).

A newcomer to Vancouver looking for a home might wonder what prices are considered below market value when looking for homes. On the flip side, a seller might wonder whether they are pricing their home below, above, or at the market expectation in order to accelerate or decelerate their time-to-sale.

The price of a house can depend on a multitude of factors, including location, size, the age of the home, and many others. Based on information about the property, predictions can be made about the selling price of a home compared to others in the area.

The goal of our project is to utilize k-nearest neighbors regression analysis to determine the relationship between price and total square footage of houses in Vancouver. Our prediction is that the total square footage can be a good indicator of home prices in Vancouver. We will do this using a [publicly available dataset from the website Kaggle,](https://www.kaggle.com/datasets/darianghorbanian/vancouver-home-price-analysis-regression), which has price data, square footage, and other details about Vancouver houses from 2017-2020. This dataset was compiled by [Darian Ghorbanian](https://www.linkedin.com/in/darianghorbanian/) during his studies at UBC Sauder School of Business, pursuing his Master of Business Analytics.


Our Predictive Question is: “Does the square footage of a house in Vancouver have an impact on its selling price?”


# Methods & Results: #


## loading data from the original source on the web ##

The dataset that we have chosen for this project is a publicly available dataset from the website Kaggle as described above. It includes price data, square footage, and other details about Vancouver houses from its respective time period.

https://www.kaggle.com/datasets/darianghorbanian/vancouver-home-price-analysis-regression 

In order to read our dataset directly from the Kaggle website, we will need to work with the Kaggle API and set it up using an authentication username and key.

First, we will import the necessary libraries to use throughout our project.

In [130]:
### importing the necessary libraries
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Simplify working with large datasets in Altair
alt.data_transformers.disable_max_rows()

# Output dataframes instead of arrays
set_config(transform_output="pandas")

Then we will install the Kaggle package, to interact with the Kaggle API as outlined in their [documentation](https://www.kaggle.com/docs/api):

In [131]:
# set up Kaggle for downloading dataset 

!pip install kaggle
import os

# note - this authentication token was created for this and has a 
# very limited permission scope

os.environ['KAGGLE_USERNAME'] = 'alexannn'
os.environ['KAGGLE_KEY'] = '134ddfd9c0609f9493f6766bad383898'

In [132]:
# download dataset
!kaggle datasets download -d darianghorbanian/vancouver-home-price-analysis-regression --unzip

Dataset URL: https://www.kaggle.com/datasets/darianghorbanian/vancouver-home-price-analysis-regression
License(s): unknown
  0%|                                               | 0.00/30.1k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 30.1k/30.1k [00:00<00:00, 12.0MB/s]


Now that we have downloaded our dataset, we can load it into a pandas data frame

In [133]:
# load our dataset into a pandas dataframe
home_prices = pd.read_csv("House sale data Vancouver.csv")
home_prices

,Number,Address,List Date,Price,Days on market,Total floor area,Year Built,Age,Lot Size
0,1,3178 GRAVELEY STREET,5/8/2020,1500000,18,2447,1946,74,5674.00
1,2,1438 E 28TH AVENUE,1/22/2020,1300000,7,2146,1982,38,3631.98
2,3,2831 W 49TH AVENUE,6/18/2019,2650000,1,3108,1929,90,9111.00
3,4,2645 TRIUMPH STREET,6/18/2019,1385000,28,2602,1922,97,4022.70
4,5,741-743 E 10TH AVENUE,11/28/2019,1590000,17,1843,1970,49,4026.00
...,...,...,...,...,...,...,...,...,...
1297,1298,65 W KING EDWARD AVENUE,8/22/2019,2630000,42,3035,1939,80,7456.00
1298,1299,3150 E 52ND AVENUE,8/17/2019,1450000,14,2282,1974,45,3993.00
1299,1300,4478 PRINCE ALBERT STREET,2/24/2020,2798000,4,3501,2016,4,3960.00
1300,1301,4038 MILLER STREET,4/5/2019,900000,194,2440,1912,107,3297.00


Table 1: Raw Housing Data

## Formatting The Data ##

#### 1. Wrangling and Cleaning Our Dataset ####

The tidy data format is a standard of compiling data, that allows for easier manipulation and modeling of the data.

It adheres to the following three principles:

- Each variable corresponds to a column.
- Each observation corresponds to a row.
- Each measurement is a cell value.

Fortunately, in our dataset, our data already meets these requirements and is therefore considered tidy. However, for the sake of simplicity, we can drop all the columns except those that are relevant to our analysis.

In [134]:
home_prices = home_prices[['Price', 'Total floor area']]
home_prices

,Price,Total floor area
0,1500000,2447
1,1300000,2146
2,2650000,3108
3,1385000,2602
4,1590000,1843
...,...,...
1297,2630000,3035
1298,1450000,2282
1299,2798000,3501
1300,900000,2440


Table 2: Price and Total Floor Area

#### 2. Splitting Our Data Into Training and Testing Sets ####

Next, we will split the data to use 75% as the training set and 25% as the testing set in order to perform k-nn regression. 

The training portion of the dataset is used to teach our model how to predict house prices from floor area data, helping it to learn the underlying patterns. 
The testing set allows us to evaluate the model’s accuracy, and its ability to generalize those patterns to unseen data.

We will set the `Price` column as the target (y) and `Total floor area` column as the input feature (X).

In [135]:
# Splitting the data
home_training, home_testing = train_test_split(
    home_prices,
    test_size=0.25,
    random_state=2000
)

# Preparing the training data for analysis
X_train = home_training[["Total floor area"]] 
y_train = home_training["Price"]  

X_test = home_testing[["Total floor area"]] 
y_test = home_testing["Price"]

## Numerical Summary of the dataset ##

Now, before we start with our analysis, it can be helpful to get a high-level summary of the dataset. In the following table, we've provided some basic numerical summaries. We've included the five-number summary for a quick overview of our data, as well as the standard deviation to represent the spread, the number of missing values, and the overall number of data points.

In [136]:
# Generating the summary table for Total floor area and Price, based on training data only
summary_table = pd.DataFrame({
    'Total floor area': [
        home_training['Total floor area'].count(),
        round(home_training['Total floor area'].mean(), 2),
        round(home_training['Total floor area'].median(), 2),
        round(home_training['Total floor area'].std(), 2),
        round(home_training['Total floor area'].min(), 2),
        round(home_training['Total floor area'].max(), 2),
        home_training['Total floor area'].isnull().sum(),
        round(home_training['Total floor area'].quantile(0.25), 2),
        round(home_training['Total floor area'].quantile(0.75), 2)
    ],
    'Price': [
        home_training['Price'].count(),
        round(home_training['Price'].mean(), 2),
        round(home_training['Price'].median(), 2),
        round(home_training['Price'].std(), 2),
        round(home_training['Price'].min(), 2),
        round(home_training['Price'].max(), 2),
        home_training['Price'].isnull().sum(),
        round(home_training['Price'].quantile(0.25), 2),
        round(home_training['Price'].quantile(0.75), 2)
    ]
}, index=['Count', 'Mean', 'Median', 'Std', 'Min', 'Max', 'Missing Values', '25th Percentile', '75th Percentile'])

# Display the summary table
summary_table

,Total floor area,Price
Count,976.00,976.00
Mean,2448.36,1774762.64
Median,2399.00,1607500.00
Std,715.83,550223.60
Min,301.00,738000.00
Max,6556.00,3000000.00
Missing Values,0.00,0.00
25th Percentile,1980.75,1332250.00
75th Percentile,2832.50,2199925.00


Table 3: Summary of Dataset

Our summary table shows that the mean total floor area of the homes in the dataset is 2,448.36 square feet, with a relatively wide range of sizes indicated by the standard deviation of 715.83 square feet. 

For the selling prices, there is a significant variation as well, with a mean price of approximately 1.77 million dollars, but with a large standard deviation of about 5.5 million dollars, which tells us that there is a wide dispersion in the home prices. 

Both the total floor area and the price have no missing values. That is helpful to us, because it means that the dataset is complete in these aspects for the training set.

## Visual Exploration of the dataset ##

As we have two quantitative variables in our dataset, with Square Footage as our explanatory variable and house selling price as our response variable, we've opted to visualize our data with a scatter plot. This choice alows us to roughly identify trends, patterns, and potential outliers at a glance.

In [137]:
#record minimum and maximum values in the prices columns
min_price = home_prices['Price'].min()
max_price = home_prices['Price'].max()

# Create the scatter plot with adjusted y-axis scale
scatter_plot = alt.Chart(home_training).mark_circle(opacity=0.5).encode(
    alt.X('Total floor area:Q', title='Total Floor Area (sq ft)'),
    alt.Y('Price:Q', scale=alt.Scale(domain=(min_price, max_price)), title='Price (CAD)').axis(format="$,.0f"),
    tooltip=['Total floor area:Q', 'Price:Q']
).properties(
    width=400,
    height=400,
    title='Figure 1: Scatter Plot of Price vs. Total Floor Area'
)

scatter_plot

alt.Chart(...)

In our preliminary data exploration using the scatter plot, we observe a concentration of data points from the bottom left to the top right that suggest a positive correlation between the total floor area and the home prices. This tells us that there is a trend indicating larger homes tend to be priced higher. 

However, there is significant variability in prices for homes of similar sizes, as indicated by the vertical spread of points at many levels of floor area. This also falls in line with what we found in our numerical summary of the dataset abobe. We can also see some potential outliers, particularly homes with high prices that do not have a proportionally large floor area. 

This might be influenced by factors not captured in the plot and our dataset such as location, home condition, or market conditions. Therefore, this graph tells us that square footage has the potential to be a good indicator for selling price, but there is more complexity in the situation involved.

From here we will move forward with our analysis to quanitfy this relationship through solid and reproducable methods. 




## Performing the data analysis ##

"To make our numerical predictions, we will be using k-nn regression. This method involves selecting the k nearest neighbors (in terms of square footage in our case) of the house we are interested in, and averaging their price to predict a selling price accordingly.

But how can we find what value of k (representing the amount of neighbors) will give us the best results? We can find an optimal value for k using cross-validation. Cross-validation involves dividing the training data into a number of smaller sets, called folds. Here we will use 5.

For each k value, we take turns using one group as a test while the others help the model learn. After testing all groups, we combine the results to see how well each k value performed overall. We can then evaluate the performance of each k value, using the root mean squared error (RMSE).

So let's get started! First, we will create a pipeline for k-nn, and use the StandardScaler to standardize the numerical values. Next, we will perform cross-validation with 5 folds using the cross_validate function. We use the negative RMSPE (to bypass sci-kit learn library's default of maximizing a score when tuning) by using "neg_root_mean_squared_error."

In [138]:
# Set the random seed for reproducibility of results
np.random.seed(2019)

# Create a pipeline that first standardizes the data, 
# then applies k-nearest neighbors regression
home_pipe = make_pipeline(
   StandardScaler(), KNeighborsRegressor())

# Perform cross-validation to evaluate the pipeline
home_cv = pd.DataFrame(
    cross_validate(
        estimator=home_pipe,
        cv=5,
        X = X_train,
        y = y_train,
        scoring = "neg_root_mean_squared_error",
        return_train_score=True
    )
)

To ensure we have the "best: value of $K$, we will test all values up to 100. First, we will create a parameter grid called param_grid, a dictionary that holds values of k between 1 to 100.

We will then tune the model using the `GridSearchCV` function, which will iterate though each $k$ value, training the model using the 5 fold method we described above, and calculates the (negative) RMSPE which it stores with its corresponding $k$ value.

By calling `best_params_` on the model, we can find the number of neighbors for the optimal $k$ value. By calling`best_score_` on the model, we can find the score for the best model.

In [153]:
np.random.seed(2019)

# Define a dictionary with parameters for the 
# KNeighborsRegressor inside the pipeline
param_grid = {
    "kneighborsregressor__n_neighbors": range(1, 101, 1),
}

# Create a GridSearchCV object to fine-tune the model
home_tuned = GridSearchCV(estimator=home_pipe, param_grid=param_grid, cv=5, scoring = "neg_root_mean_squared_error", n_jobs=-1)

# Fit the GridSearchCV to the training data and retrieve the cross-validation results
home_results = pd.DataFrame(home_tuned.fit(home_training[["Total floor area"]], home_training["Price"]).cv_results_) 

#display our resulting dataframe
home_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsregressor__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003425,0.000280,0.002463,0.000269,1,{'kneighborsregressor__n_neighbors': 1},-555528.754220,-629172.956999,-631816.083758,-603687.286602,-597927.594062,-603626.535128,27543.233182,100
1,0.003224,0.000052,0.002274,0.000092,2,{'kneighborsregressor__n_neighbors': 2},-515797.485603,-543752.558005,-521718.867047,-552993.736387,-519001.936253,-530652.916659,14879.242241,99
2,0.003413,0.000368,0.002274,0.000082,3,{'kneighborsregressor__n_neighbors': 3},-491882.991900,-515478.954608,-475642.828958,-510114.475759,-505044.002217,-499632.650688,14323.585362,98
3,0.003225,0.000027,0.002249,0.000010,4,{'kneighborsregressor__n_neighbors': 4},-487744.125636,-491257.379900,-463348.450025,-509884.917520,-498289.497365,-490104.874089,15369.097618,97
4,0.003196,0.000028,0.002251,0.000010,5,{'kneighborsregressor__n_neighbors': 5},-479640.466931,-457903.045593,-450008.998877,-494313.504613,-480274.407790,-472428.084761,16162.592728,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.003149,0.000005,0.003603,0.000033,96,{'kneighborsregressor__n_neighbors': 96},-453364.294804,-428882.289667,-442703.858362,-422039.708518,-441238.453027,-437645.720876,11008.282280,12
96,0.003161,0.000019,0.003616,0.000024,97,{'kneighborsregressor__n_neighbors': 97},-453351.174185,-429044.717785,-442249.875264,-421419.413238,-441227.120656,-437458.460226,11115.582815,7
97,0.003169,0.000012,0.003647,0.000030,98,{'kneighborsregressor__n_neighbors': 98},-453427.478077,-429334.082630,-442352.584861,-420931.830097,-440950.258406,-437399.246814,11227.396183,5
98,0.003181,0.000015,0.003721,0.000045,99,{'kneighborsregressor__n_neighbors': 99},-453510.868724,-429000.306908,-442043.959512,-420943.624430,-441079.212137,-437315.594342,11278.249619,3


Table 4: Grid Search Results

In [154]:
# Display the optimal value for K
home_min = home_tuned.best_params_
home_min

{'kneighborsregressor__n_neighbors': 63}

In [155]:
# Display the optimal score found
home_best_RMSPE = -home_tuned.best_score_
home_best_RMSPE

437251.0465385521

After the tuning and the training, we can see that the optimal K value yields an RMSPE value of around $437,251. This is still fairly high, indicating a more complex analysis is required. We will delve deeper into interperting this value in the discussion section below.

## Creating a visualization of the analysis ##

To provide a clear visual representation of how the number of neighbors affects the K-Nearest Neighbors regression model's accuracy, we plotted the Root Mean Squared Prediction Error (RMSPE) against varying values of `n_neighbors`. This graph is often insturmental in identifying the optimal number of neighbors to use in order to strike a balance between model complexity and predictive performance.

In [151]:
# Converting our mean test scores to positive for easy visualization
home_results['mean_test_score'] = home_results['mean_test_score'].abs()

# Plotting the test score against each K value
chart = alt.Chart(home_results).mark_line(point=True).encode(
    x=alt.X('param_kneighborsregressor__n_neighbors', title='Number of Neighbors (n_neighbors)'),
    y=alt.Y('mean_test_score', title='Root Mean Squared Prediction Error (RMSPE)'),
    tooltip=['param_kneighborsregressor__n_neighbors', 'mean_test_score']
).properties(
    title='Figure 2: Model Performance vs. Number of Neighbors',
    width=600,
    height=400
)

#display our chart
chart


alt.Chart(...)

The graph depicting the performance of the K-Nearest Neighbors regression model shows a sharp decline in the Root Mean Squared Prediction Error (RMSPE) as the number of neighbors increases from 1, suggesting that a single neighbor leads to overfitting. The RMSPE stabilizes shortly after, indicating that adding more neighbors beyond this point does not significantly improve the model's predictive accuracy. This stabilization occurs at around 5 neighbors, which likely represents an optimal balance between model complexity and prediction error, hinting that a relatively simple model is sufficient to capture the trends in the Vancouver housing data. However, this is very different from the result we previously achieved with our cross-validation method. Why is that? There are several possible reasons, and while a full investigation is beyond the scope of this course, here are some likely culprits:


1. Noise and Variability: 
Because of the large variability in our prices as indicated in our pre-analysis data exploration above, a smaller k value means that the averaging of the data is more sensitive to variation. Larger k values can balance those variations out, leading to better results not captured by the graph above.
2. Complexity of the model:
3. Smaller k values can also lead to more complex models, which can perform well on training data but do not generalize well. The grid search with cross-validation finds a better balance that can predict new data. We can see this once we plot the Knn regression fit graph below."

In [152]:
# Use the trained pipeline to predict over a range of X values for visualization
X_vis = pd.DataFrame(np.linspace(X_train['Total floor area'].min(), X_train['Total floor area'].max(), 200), columns=['Total floor area'])
home_pipe.set_params(**home_min)  # Set the best found parameters
home_pipe.fit(X_train[['Total floor area']], y_train)  # Fit the pipeline with the training data
y_pred = home_pipe.predict(X_vis)

# Create DataFrames for plotting
train_df = pd.DataFrame({'Total floor area': X_train['Total floor area'], 'Price': y_train})
test_df = pd.DataFrame({'Total floor area': X_test['Total floor area'], 'Price': y_test})
pred_df = pd.DataFrame({'Total floor area': X_vis['Total floor area'], 'Price': y_pred})

# Create Single DataFrame for plotting
train_df["Type"] = "Train"
test_df["Type"] = "Test"
pred_df["Type"] = "Prediction"
three_in_one = pd.concat([train_df, test_df, pred_df], ignore_index=True)

# Map colors of each subplot
scale = alt.Scale(domain=['Train', 'Test', 'Prediction'], range=['blue', 'green', 'red'])

# Base chart for the training points
train_points = alt.Chart(train_df).mark_circle(size=60, opacity=0.5).encode(
    x='Total floor area:Q',
    y='Price:Q',
    color=alt.Color('Type:N', scale=scale)
)

# Points for the test data
test_points = alt.Chart(test_df).mark_circle(size=60, opacity=0.5).encode(
    x='Total floor area:Q',
    y='Price:Q',
    color=alt.Color('Type:N', scale=scale)
)

# Line for the prediction
prediction_line = alt.Chart(pred_df).mark_line().encode(
    x='Total floor area:Q',
    y = alt.Y('Price:Q', title='Price (CAD)').axis(format="$,.0f"),
    color=alt.Color('Type:N', scale=scale)
)

# Combine the charts
chart = (train_points + test_points + prediction_line).properties(
    width=600,
    height=400,
    title='Figure 3: KNN Regression Fit'
)

# Set chart axis titles
chart.layer[0].encoding.y.title = 'Price (CAD)'
chart.layer[0].encoding.x.title = 'Total floor area (sq ft)'

# Display the chart
chart

alt.LayerChart(...)

Finally, we can graph our training data, our testing data, and our KNN regression model. We can see that the model roughly follows the trend of the housing prices, despite the large variablity in home pricing. The model seems to underfit at higher floor areas, and very low floor areas, where the actual prices have a broader range, indicating that we will need more data points to improve predictions in those regions.

# Discussion: #


In completing the KNN regression analysis of the data and generating a predictive model, we found that the price of houses in Vancouver roughly increases with total square footage, demonstrating a weak positive relationship. Our predictive model reflected this relationship to a certain extent with decreased accuracy near the endpoints.

During the cross-validation portion of our analysis, we found that the optimal K value for knn regression in our dataset is 63. If our dataset is a good representative sample of the population, the actual K value for the population is likely not too far off.

However, our optimal K value produced an RMSPE value of around $437,251. We found in our preliminary data analysis that there is wide variability in our pricing, indicating that square footage alone is not sufficient to get a precise prediction. That is reflected in our RMSPE value, which indicates that more explanatory variables might be needed for better analysis, i.e., construction material, location, year built, etc.


Our predictive model did not fully align with our expectations. We predicted that the relationship between square footage and price would be positive and that predicted prices would increase linearly with total floor area. Instead, the prediction line plateaus at either end, seemingly indicating that after a certain point, houses with increasing square footage would be similar in price, and vice versa. However, as mentioned above, this could also be an indication that we need more data points. Further exploration is needed to confirm.


The findings of a project like this are impactful in a multitude of ways. They could help a person looking to sell their home make an informed decision on the best price to list their property at based on the current housing market. They could also provide valuable insight for a buyer looking for a property in Vancouver who wants to know they're buying a house at a reasonable price. Additionally, these findings could have an impact on investments, policies, and urban planning in Vancouver.


This project could lead to further questions such as:


What is the biggest determining factor for the price of a home?
How does the relationship between square footage and house price in Vancouver compare to other cities with similar housing markets in Canada?
During the cross validation portion of our analysis, we found that the optimal K value for knn regression in our dataset is 63. If our dataset is a good representive sample of the population, the actualy K value for the population is likely not too far off. 

However, our optimal K value produced an RMSPE value of around $437,251. We found in our preliminary data analysis that there is a wide variability in our pricing, indicating that square footage alone is not sufficient to get a precise prediction. That is reflected in our RMSPE value, which indicates that more explantory variables might be needed for better analysis, i.e. construction material, location, year built, ect.

Our predictive model did not fully align with our expectations, as we predicted that the relationship between square footage and price would be positive, and therefore predicted prices would increase linearly with total floor area. Instead the prediction line plateaus at either end, seemingly indicating that after a certain point, houses with increasing square footage would be similar in price, and vice versa. However, as mentioned above, this could also be an indication that we need more data points. Further exploration is needed to confirm.

The findings of a project like this are impactful in a multitude of ways. They could help a person looking to sell their home make an informed decision on the best price to list their property at based on the current housing market, or provide valuable insight for a buyer looking for a property in Vancouver who wants to know that they're buying a house at a reasonable price. Additionally, these findings could have an impact on investments, policies, and urban planning in Vancouver.

This project could lead to further questions such as:

- What are some other factors that influence the price of housing in Vancouver?
- What is the biggest determining factor for the price of a home?
- How does the relationship between square footage and house price in Vancouver compare to other cities with similar housing markets in Canada?

# References: #


1. Ghorbanian, D. (2023, April). Vancouver Home Price Analysis – Regression, Version Retrieved March 9, 2024 from https://www.kaggle.com/datasets/darianghorbanian/vancouver-home-price-analysis-regression/data
   
2. Timbers, T., Campbell, T., Lee, M., Ostblom, J., & Heagy, L. (n.d.). Data Science: A First Introduction (Python Edition) [Online Book]. Retrieved from https://python.datasciencebook.ca/index.html
   
3. Kwan, S. (2019). Vancouver housing ranked world’s second-least affordable market. BNN Bloomberg. https://www.bnnbloomberg.ca/hong-kong-housing-ranked-world-s-least-affordable-for-9th-year-1.1201263

